In [1]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
import re
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/rkumar-bengaluru/data-science/main/16-Projects/zuel/data/2001_all_materials.csv')
df['Description'] = df['Description'].astype(str)

In [3]:
# initialize test data
test = df['Description']
test = test[:]

In [4]:
# word 2 index
nltk_stop_words = stopwords.words ('english')
tokenizer = RegexpTokenizer(r'\w+')
doc2index = {}

def word2index(corpus):
    idx = 0
    w2i = {}
    all_docs_as_int = []
    j = 0
    for doc in corpus:
        doc_as_int = []
        wordsindoc = tokenizer.tokenize(doc.lower())
        doc2index[doc] = j
        j += 1
        for word in wordsindoc:
            if word not in nltk_stop_words:
                # check if this word exist in dictionary
                if word not in w2i:
                    w2i[word] = idx # dettol as 0
                    idx += 1

                doc_as_int.append(w2i[word])
                
        all_docs_as_int.append(doc_as_int)
    return w2i,all_docs_as_int

def index2word(w2i):
    i2w = []
    for k,v in w2i.items():
        i2w.append(k)
    return i2w

def sample_words(w2i,de):
    testwords = []
    for k,v in w2i.items():
        if k.startswith(de):
            testwords.append(k)
    return testwords

def all_words(w2i):
    testwords = []
    for k,v in w2i.items():
        testwords.append(k)
    return testwords

In [5]:
#print('test_data\n',test)
w2i,docs_as_int = word2index(test)
# words starting with de
testwords = sample_words(w2i,'de')
#testwords = testwords[:5]
testwords.remove('depotamp250mg1x20')
testwords.remove('defensespf5030ml')
testwords.remove('def')
testwords.remove('dermovateoint100gm')
testwords.remove('de')
testwords.remove('dermovateoint15gm')
testwords.remove('depottabs60mg7x15')
testwords.remove('deodorant50ml')
testwords.remove('desferalvial500mg10')
testwords.remove('defensecream50ml')
testwords.remove('dexcream5g')
testwords.remove('dermgel1')
print('testwords\n',testwords)

testwords
 ['dermaveen', 'depiderm', 'demaquillantepnm', 'deluxe9', 'dettol', 'dermaceutic', 'depo', 'deep', 'derma', 'decolgen', 'dermadrate', 'dentalcrm', 'deluxe3', 'dew', 'deense', 'dermgel', 'delilah', 'devil', 'deluxe', 'dermovatecream100gm', 'devices', 'dextrose', 'detangler', 'demaquillant', 'dermovatecream15gm', 'dental', 'deodorant', 'dermareen', 'dermatix', 'dermacontrol', 'dermaid', 'defense', 'detol', 'dermoprotective', 'deluxe5', 'debtin', 'derived', 'deluxe1', 'deluxe7', 'deluxe4']


In [6]:
import numpy as np
import string

In [7]:
np.random.seed(1234)

In [8]:
def add2dict(d,k,v):
    if k not in d:
        d[k] = []
    d[k].append(v)

In [9]:
def markov_second_order(words):
    initial = {} # dictionary of pharases
    first_order_markov = {} # second word only.
    second_order_markov = {} # from second word onwards
    for word in words:
        T = len(word)
        for i in range(T):
            t = word[i]
            if i ==0:
                # first character
                initial[t] = initial.get(t,0.) + 1
            else:
                t_1 = word[i -1]
                if i == T-1:
                    # last character
                    add2dict(second_order_markov, (t_1,t),'END')
                if i == 1:
                    # second character
                    add2dict(first_order_markov,t_1,t)
                else:
                    # all the remaining.
                    t_2 = word[i-2]
                    add2dict(second_order_markov, (t_2,t_1),t)
    return initial,first_order_markov,second_order_markov

In [10]:
initial,first_order_markov,second_order_markov = markov_second_order(testwords)
print('testwords\n',testwords)
print('initial\n',initial)
print('first_order_markov\n',first_order_markov)
print('second_order_markov\n',second_order_markov)

testwords
 ['dermaveen', 'depiderm', 'demaquillantepnm', 'deluxe9', 'dettol', 'dermaceutic', 'depo', 'deep', 'derma', 'decolgen', 'dermadrate', 'dentalcrm', 'deluxe3', 'dew', 'deense', 'dermgel', 'delilah', 'devil', 'deluxe', 'dermovatecream100gm', 'devices', 'dextrose', 'detangler', 'demaquillant', 'dermovatecream15gm', 'dental', 'deodorant', 'dermareen', 'dermatix', 'dermacontrol', 'dermaid', 'defense', 'detol', 'dermoprotective', 'deluxe5', 'debtin', 'derived', 'deluxe1', 'deluxe7', 'deluxe4']
initial
 {'d': 40.0}
first_order_markov
 {'d': ['e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e']}
second_order_markov
 {('d', 'e'): ['r', 'p', 'r', 'm', 'l', 't', 'r', 'p', 'e', 'r', 'c', 'r', 'n', 'l', 'w', 'e', 'r', 'l', 'v', 'l', 'r', 'v', 'x', 't', 'm', 'r', 'n', 'o', 'r', 'r', 'r', 'r', 'f', 't', 'r', 'l', 'b', 'r', 'l', 'l', 'l'], ('e', 'r

In [11]:
# normalize distribution
def normalize_initial_distribution(initial):
    initial_total = sum(initial.values())
    for t,c in initial.items():
        initial[t] = c/initial_total

def normalize_markov_order_distribution(ts):
    d = {}
    n = len(ts)
    for t in ts:
        d[t] = d.get(t,0.) + 1
    for t,c in d.items():
        d[t] = c/n
    return d

In [12]:
initial,first_order_markov,second_order_markov = markov_second_order(testwords)
print('initial\n',initial)
print('first_order_markov\n',first_order_markov)
print('second_order_markov\n',second_order_markov)

initial
 {'d': 40.0}
first_order_markov
 {'d': ['e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e']}
second_order_markov
 {('d', 'e'): ['r', 'p', 'r', 'm', 'l', 't', 'r', 'p', 'e', 'r', 'c', 'r', 'n', 'l', 'w', 'e', 'r', 'l', 'v', 'l', 'r', 'v', 'x', 't', 'm', 'r', 'n', 'o', 'r', 'r', 'r', 'r', 'f', 't', 'r', 'l', 'b', 'r', 'l', 'l', 'l'], ('e', 'r'): ['m', 'm', 'm', 'm', 'm', 'm', 'm', 'END', 'm', 'm', 'm', 'm', 'm', 'm', 'i'], ('r', 'm'): ['a', 'END', 'a', 'a', 'a', 'END', 'g', 'o', 'o', 'a', 'a', 'a', 'a', 'o'], ('m', 'a'): ['v', 'q', 'c', 'END', 'd', 'q', 'r', 't', 'c', 'i'], ('a', 'v'): ['e'], ('v', 'e'): ['e', 'END', 'd'], ('e', 'n'): ['END', 'END', 't', 's', 't', 'END', 's'], ('e', 'e'): ['n', 'p', 'n', 'n'], ('e', 'p'): ['i', 'n', 'o', 'END'], ('p', 'i'): ['d'], ('i', 'd'): ['e', 'END'], ('e', 'm'): ['a', 'a'], ('a', 'q'): ['u', 'u'

In [13]:
def normalize(initial,first_order,second_order):
    normalize_initial_distribution(initial)
    for t_1,ts in first_order.items():
        first_order[t_1] = normalize_markov_order_distribution(ts)
    for t_1,ts in second_order.items():
        second_order[t_1] = normalize_markov_order_distribution(ts)

normalize(initial,first_order_markov,second_order_markov)

print('test_data\n',testwords)
print('initial\n',initial)
print('first_order_markov\n',first_order_markov)
print('second_order_markov\n',second_order_markov)
second_order_markov

test_data
 ['dermaveen', 'depiderm', 'demaquillantepnm', 'deluxe9', 'dettol', 'dermaceutic', 'depo', 'deep', 'derma', 'decolgen', 'dermadrate', 'dentalcrm', 'deluxe3', 'dew', 'deense', 'dermgel', 'delilah', 'devil', 'deluxe', 'dermovatecream100gm', 'devices', 'dextrose', 'detangler', 'demaquillant', 'dermovatecream15gm', 'dental', 'deodorant', 'dermareen', 'dermatix', 'dermacontrol', 'dermaid', 'defense', 'detol', 'dermoprotective', 'deluxe5', 'debtin', 'derived', 'deluxe1', 'deluxe7', 'deluxe4']
initial
 {'d': 1.0}
first_order_markov
 {'d': {'e': 1.0}}
second_order_markov
 {('d', 'e'): {'r': 0.34146341463414637, 'p': 0.04878048780487805, 'm': 0.04878048780487805, 'l': 0.1951219512195122, 't': 0.07317073170731707, 'e': 0.04878048780487805, 'c': 0.024390243902439025, 'n': 0.04878048780487805, 'w': 0.024390243902439025, 'v': 0.04878048780487805, 'x': 0.024390243902439025, 'o': 0.024390243902439025, 'f': 0.024390243902439025, 'b': 0.024390243902439025}, ('e', 'r'): {'m': 0.866666666666666

{('d', 'e'): {'r': 0.34146341463414637,
  'p': 0.04878048780487805,
  'm': 0.04878048780487805,
  'l': 0.1951219512195122,
  't': 0.07317073170731707,
  'e': 0.04878048780487805,
  'c': 0.024390243902439025,
  'n': 0.04878048780487805,
  'w': 0.024390243902439025,
  'v': 0.04878048780487805,
  'x': 0.024390243902439025,
  'o': 0.024390243902439025,
  'f': 0.024390243902439025,
  'b': 0.024390243902439025},
 ('e', 'r'): {'m': 0.8666666666666667,
  'END': 0.06666666666666667,
  'i': 0.06666666666666667},
 ('r', 'm'): {'a': 0.5714285714285714,
  'END': 0.14285714285714285,
  'g': 0.07142857142857142,
  'o': 0.21428571428571427},
 ('m', 'a'): {'v': 0.1,
  'q': 0.2,
  'c': 0.2,
  'END': 0.1,
  'd': 0.1,
  'r': 0.1,
  't': 0.1,
  'i': 0.1},
 ('a', 'v'): {'e': 1.0},
 ('v', 'e'): {'e': 0.3333333333333333,
  'END': 0.3333333333333333,
  'd': 0.3333333333333333},
 ('e', 'n'): {'END': 0.42857142857142855,
  't': 0.2857142857142857,
  's': 0.2857142857142857},
 ('e', 'e'): {'n': 0.75, 'p': 0.25},


In [14]:
def next_character(d):
    probs = []
    chars = []
    
    cumulative = 0
    for t,p in d.items():
        probs.append(p)
        chars.append(t)
   
    #print(chars)
    return chars[:1]

In [15]:
def copy_chars(inputs):
    res = []
    for w in inputs:
        res.append(w)
    return res

In [16]:
def generate(initial):
    suggestions = []
    for word in initial:
        length = len(word)
        w0 = word[length-2]
        w1 = word[length-1]
        w = copy_chars(word)
        w2 = next_character(second_order_markov[w0,w1])
        for c in w2:
            if c != 'END':
                tmp = copy_chars(w)
                tmp.append(c)
                suggestions.append(''.join(tmp))
            if c == 'END':
                tmp = copy_chars(w)
                suggestions.append(''.join(tmp))
    return suggestions

In [17]:
def generate_word(suggestions,length):
    for i in range(length):
        suggestions = generate(suggestions)
    return suggestions

In [29]:
suggestions = ['det']
suggestions = generate_word(suggestions,7)
print('length of suggestion\n',len(suggestions))
print('suggestions',suggestions)

length of suggestion
 1
suggestions ['dettol']
